In [1]:
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt
import os 
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/tmp/ipykernel_1478/3924256960.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
/opt/conda/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For

## Espacios públicos gdl

In [2]:
schema = 'espacios_publicos'
table = 'ep_amg'
gdf = aup.gdf_from_db(table,schema)
gdf.head()

,Fuente,Tipo,Uso,geometry
0,"OpenStreetMap, [Online]. Disponible: http://ww...",Parque,Uso recreativo,"POLYGON ((672507.028 2272132.201, 672520.126 2..."
1,"OpenStreetMap, [Online]. Disponible: http://ww...",Parque,Uso recreativo,"POLYGON ((672520.126 2272170.353, 672507.028 2..."
2,"OpenStreetMap, [Online]. Disponible: http://ww...",Parque,Uso recreativo,"POLYGON ((672494.325 2272095.205, 672480.366 2..."
3,"OpenStreetMap, [Online]. Disponible: http://ww...",Parque,Uso recreativo,"POLYGON ((672480.366 2272060.731, 672494.325 2..."
4,"OpenStreetMap, [Online]. Disponible: http://ww...",Parque,Uso recreativo,"POLYGON ((672468.717 2272020.613, 672456.895 2..."


In [3]:
gdf.shape

(8499, 4)

## Hexágonos con info de humedad, vegetación y temperatura

select {tempe}.hex_id, {tempe}.geometry, {tempe}.city, {tempe}.res,

temperature_std, temperature_median, temperature_max, temperature_min, temperature_diff, temperature_tend,temp_diff_mean,

ndvi_mean, ndvi_std, ndvi_median, ndvi_max, ndvi_min, ndvi_diff, ndvi_tend,

ndmi_mean, ndmi_std, ndmi_median, ndmi_max, ndmi_min, ndmi_diff, ndmi_tend

from raster_analysis.temperature_analysis_hex tempe

inner join raster_analysis.ndmi_analysis_hex ndmi

on tempe.hex_id = ndmi.hex_id 

inner join raster_analysis.ndvi_analysis_hex ndvi

on tempe.hex_id = ndvi.hex_id 

where tempe.city  = 'Guadalajara' and tempe.res = 11

In [4]:
hex_schema = "raster_analysis"
hex_table = "temperature_analysis_hex"
hex_tabl = "ndvi_analysis_hex"
hex_tablee = "ndmi_analysis_hex"
tempe = "tempe"
ndmi = "ndmi"
ndvi = "ndvi"
hex_id = "hex_id"
c = 'Guadalajara'
n = '11'
query = f'SELECT {tempe}.hex_id, {tempe}.geometry, {tempe}.city, {tempe}.res, temperature_std, temperature_median, temperature_max, temperature_min, temperature_diff, temperature_tend,temp_diff_mean,ndvi_mean, ndvi_std, ndvi_median, ndvi_max, ndvi_min, ndvi_diff, ndvi_tend,ndmi_mean, ndmi_std, ndmi_median, ndmi_max, ndmi_min, ndmi_diff, ndmi_tend FROM {hex_schema}.{hex_table} {tempe} INNER JOIN {hex_schema}.{hex_tablee} {ndmi} ON {tempe}.{hex_id} = {ndmi}.{hex_id} INNER JOIN {hex_schema}.{hex_tabl} {ndvi} ON {tempe}.{hex_id} = {ndvi}.{hex_id} WHERE {tempe}.\"city\" = \'{c}\' AND {tempe}.\"res\" = {n}'
int_gdl = aup.gdf_from_query(query, geometry_col='geometry')
int_gdl.head(10)

,hex_id,geometry,city,res,temperature_std,temperature_median,temperature_max,temperature_min,temperature_diff,temperature_tend,...,ndvi_min,ndvi_diff,ndvi_tend,ndmi_mean,ndmi_std,ndmi_median,ndmi_max,ndmi_min,ndmi_diff,ndmi_tend
0,8b49ab4aa749fff,"POLYGON ((-103.50105 20.79588, -103.50088 20.7...",Guadalajara,11,7.278205,33.812500,46.081250,24.193750,21.887500,0.010096,...,0.090805,0.196523,-0.001515,-0.041406,0.041822,-0.046118,0.022907,-0.094032,0.116939,0.000115
1,8b49ab4b4328fff,"POLYGON ((-103.42666 20.71275, -103.42649 20.7...",Guadalajara,11,6.068860,35.958332,45.712500,27.625000,18.087500,0.016463,...,0.194372,0.224525,0.000391,0.070217,0.058914,0.069816,0.170506,-0.002577,0.173083,-0.000248
2,8b49ab499c56fff,"POLYGON ((-103.44482 20.85017, -103.44464 20.8...",Guadalajara,11,9.920609,35.744793,52.031250,21.237500,30.793750,-0.011378,...,0.120812,0.304957,-0.001107,-0.019183,0.083258,-0.038235,0.149457,-0.124775,0.274233,0.000644
3,8b49ab4b500afff,"POLYGON ((-103.44053 20.72395, -103.44035 20.7...",Guadalajara,11,5.886899,33.083332,41.958333,24.925000,17.033333,0.010417,...,0.183949,0.149432,-0.002140,0.032659,0.047775,0.021888,0.130871,-0.018883,0.149754,-0.000361
4,8b49ab4b51aafff,"POLYGON ((-103.43812 20.72080, -103.43794 20.7...",Guadalajara,11,7.586168,34.589286,46.357143,23.214286,23.142857,0.019921,...,0.124737,0.306174,-0.000613,-0.056733,0.101499,-0.081554,0.124490,-0.168416,0.292906,0.000992
5,8b49ab4b519dfff,"POLYGON ((-103.43712 20.72170, -103.43695 20.7...",Guadalajara,11,8.180542,35.227679,48.771429,24.328571,24.442857,0.017634,...,0.111357,0.432749,-0.001167,-0.051283,0.139147,-0.076312,0.191281,-0.213301,0.404582,0.002151
6,8b49ab4b516afff,"POLYGON ((-103.44442 20.72132, -103.44424 20.7...",Guadalajara,11,6.192426,30.984375,40.656250,22.262500,18.393750,0.041607,...,0.287053,0.190850,-0.002254,0.135923,0.028887,0.133809,0.183843,0.109312,0.074532,-0.000415
7,8b498c864b81fff,"POLYGON ((-103.33105 20.77061, -103.33087 20.7...",Guadalajara,11,6.398005,31.531250,43.641667,23.916667,19.725000,0.024617,...,0.174199,0.536310,-0.005124,0.067777,0.095948,0.038590,0.233523,-0.051079,0.284602,-0.000863
8,8b49ab4a2135fff,"POLYGON ((-103.46108 20.73176, -103.46091 20.7...",Guadalajara,11,8.439998,36.880953,50.328571,25.192857,25.135714,-0.042277,...,0.242814,0.439856,-0.001315,0.064440,0.124943,0.038693,0.281097,-0.079314,0.360411,0.002004
9,8b49ab4a211dfff,"POLYGON ((-103.46132 20.73399, -103.46114 20.7...",Guadalajara,11,6.357602,31.609375,40.987500,22.306250,18.681250,0.002799,...,0.295501,0.240420,-0.003290,0.135164,0.036952,0.137289,0.193261,0.097769,0.095492,-0.001074


In [7]:
int_gdl = int_gdl.set_crs("EPSG:4326")
gdf = gdf.set_crs("EPSG:4326", allow_override=True)

In [8]:
inter = gpd.sjoin(int_gdl, gdf, op='intersects')
inter.head()

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,hex_id,geometry,city,res,temperature_std,temperature_median,temperature_max,temperature_min,temperature_diff,temperature_tend,...,ndmi_std,ndmi_median,ndmi_max,ndmi_min,ndmi_diff,ndmi_tend,index_right,Fuente,Tipo,Uso
